In [28]:
import escher
import cobra

In [43]:
#Vemos los mapas disponibles en escher
escher.list_available_maps()

[{'map_name': 'e_coli_core.Core metabolism', 'organism': 'Escherichia coli'},
 {'map_name': 'iJO1366.Central metabolism', 'organism': 'Escherichia coli'},
 {'map_name': 'iJO1366.Fatty acid beta-oxidation',
  'organism': 'Escherichia coli'},
 {'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)',
  'organism': 'Escherichia coli'},
 {'map_name': 'iJO1366.Nucleotide and histidine biosynthesis',
  'organism': 'Escherichia coli'},
 {'map_name': 'iJO1366.Nucleotide metabolism', 'organism': 'Escherichia coli'},
 {'map_name': 'RECON1.Amino acid metabolism (partial)',
  'organism': 'Homo sapiens'},
 {'map_name': 'RECON1.Carbohydrate metabolism', 'organism': 'Homo sapiens'},
 {'map_name': 'RECON1.Glycolysis TCA PPP', 'organism': 'Homo sapiens'},
 {'map_name': 'RECON1.Inositol retinol metabolism',
  'organism': 'Homo sapiens'},
 {'map_name': 'RECON1.Tryptophan metabolism', 'organism': 'Homo sapiens'},
 {'map_name': 'iMM904.Central carbon metabolism',
  'organism': 'Saccharomyces cerevisiae'}

In [29]:
b = escher.Builder(map_name='e_coli_core.Core metabolism')
b.display_in_notebook()

###### Cargamos el modelo de E. coli iML1515, optimizamos crecimiento con los parametros por default

In [30]:
model = cobra.io.load_json_model('../vfajardo/iML1515.json')
solution = model.optimize()
print('Growth rate: %.2f' % solution.objective_value)

Growth rate: 0.88


###### con la funcion model.summary podemos ver las entradas y salidas del modelo

In [31]:
model.summary()

IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      22.1       h2o_e  47.2   BIOMASS_Ec_i...  0.877
glc__D_e  10         co2_e  24
nh4_e      9.47      h_e     8.06
pi_e       0.846
so4_e      0.221
k_e        0.171
fe2_e      0.0141
mg2_e      0.00761
ca2_e      0.00456
cl_e       0.00456
cu2_e      0.000622
mn2_e      0.000606
zn2_e      0.000299
ni2_e      0.000283


###### Visualizar los flujos de la solucion del modelo (solution.fluxes) en el mapa de E. coli core metabolism

In [32]:
b = escher.Builder(map_name='e_coli_core.Core metabolism',
                   reaction_data=dict(solution.fluxes),
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'value', 'value': 0.1, 'color': '#cccccc', 'size': 8},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # absolute value and no text for data
                   reaction_styles=['size', 'color', 'abs'],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook()

###### Cambiamos el modelo para simular un crecimiento anaerobico

In [34]:
#creamos una variable anmodel para diferenciar los dos modelos
anmodel = cobra.io.load_json_model('../vfajardo/iML1515.json')
anmodel.reactions.EX_o2_e.lower_bound = 0
ansolution = anmodel.optimize()
print('Growth rate: %.2f' % ansolution.objective_value)

Growth rate: 0.16


In [35]:
#model.summary nos da el resumen de las entradas y salidas
anmodel.summary()

IN FLUXES            OUT FLUXES       OBJECTIVES
-------------------  ---------------  ----------------------
glc__D_e  10         h_e     28.6     BIOMASS_Ec_i...  0.158
h2o_e      4.59      for_e   18.2
nh4_e      1.7       ac_e     8.83
pi_e       0.152     etoh_e   8.75
co2_e      0.0578    succ_e   0.0526
so4_e      0.0397
k_e        0.0308
mg2_e      0.00137
fe2_e      0.0013
fe3_e      0.00123
ca2_e      0.00082
cl_e       0.00082
cu2_e      0.000112
mn2_e      0.000109
zn2_e      5.37e-05
ni2_e      5.09e-05


In [36]:
#visualizamos
c = escher.Builder(map_name='e_coli_core.Core metabolism',
                   reaction_data=dict(ansolution.fluxes),
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'value', 'value': 0.1, 'color': '#cccccc', 'size': 8},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # absolute value and no text for data
                   reaction_styles=['size', 'color', 'abs'],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
c.display_in_notebook()

###### la variable "anmodel" ya tiene un modelo con el consumo de oxigeno en zero, sabemos que en condiciones anaerobicas E. coli consume glucosa a una tasa mas alta. Vamos a meter una restriccion para forzar al modelo a usar mas glucosa a valores reportados en la literatura (McCloskey et al 2014)

In [41]:
anmodel.reactions.EX_glc__D_e.lower_bound = -14
anmodel.reactions.EX_glc__D_e.upper_bound = -14
ansolution = anmodel.optimize()
print('Growth rate: %.2f' % ansolution.objective_value)

Growth rate: 0.24


In [42]:
#visualizamos
d = escher.Builder(map_name='e_coli_core.Core metabolism',
                   reaction_data=dict(ansolution.fluxes),
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'value', 'value': 0.1, 'color': '#cccccc', 'size': 8},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # absolute value and no text for data
                   reaction_styles=['size', 'color', 'abs'],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
d.display_in_notebook()

###### importamos un data set de RNAseq comparando crecimiento aerobio vs anaerobio en E. coli como un dataframe (pandas)

In [37]:
import pandas as pd
rnaseq = pd.read_table('./S6_RNAseqAerobictoAnaer.csv', sep=',', header=0, index_col=0 )
#S6_RNA-seq _aerobic_to _anaer.csv
rnaseq.head()

,aerobic,anaerobic
b0001,4895.513333,8567.383333
b0002,1600.476667,2327.310000
b0003,1201.873333,2042.080000
b0004,1417.103333,1669.080000
b0005,129.385967,137.558333


###### Escher usa un arreglo de diccionarios, cada "condicion" es un diccionario en este caso

In [38]:
rnaseq_array = [dict(zip(rnaseq.index, x)) for x in rnaseq.values.T]

###### Cargamos los datos en el mapa de Escher, usaremos "log2_fold" como estilo, ver la documentacion de Escher para otros

In [39]:
b = escher.Builder(map_name='e_coli_core.Core metabolism',
                   gene_data=rnaseq_array,
                   reaction_compare_style='log2_fold',
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': 'green', 'size': 25},
                                   {'type': 'value', 'value': 0, 'color': '#cccccc', 'size': 8},
                                   {'type': 'max', 'color': 'red', 'size': 25}],
                   # absolute value and no text for data
                   reaction_styles=['size', 'color', 'text'],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook()